In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableBranch
from langchain.agents import Tool
from langchain.agents import initialize_agent

In [2]:
import pinecone

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
from dotenv import dotenv_values

config = dotenv_values("../.env")

In [4]:
PINECONE_API = config['PINECONE_API']
PINECONE_ENV = config['PINECONE_ENV']

In [5]:
# find API key in console at app.pinecone.io
YOUR_API_KEY = PINECONE_API
# find ENV (cloud region) next to API key in console
YOUR_ENV = PINECONE_ENV

index_name = 'llm-recommender-system'
pinecone.init(
    api_key=YOUR_API_KEY,
    environment=YOUR_ENV
)


In [7]:
from FlagEmbedding import FlagModel

model = FlagModel('BAAI/bge-large-en-v1.5', 
                query_instruction_for_retrieval="Represent this sentence for searching relevant passages: ",
                use_fp16=True)

embed = lambda x: model.encode(x).tolist()            

In [8]:
from langchain.vectorstores import Pinecone

text_field = "description"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embed, text_field
)

/Users/jonathan/miniforge3/envs/ds/lib/python3.8/site-packages/langchain/vectorstores/pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [17]:
user_query = '''
Name: Jonathan
Age: 29
Location: New York, USA
Reading Preferences: Fiction, Historical Novels
'''

In [10]:
llm = ChatOpenAI(
    model_name='gpt-4-1106-preview',
    temperature=0.0
)

In [15]:
def format_docs_title(docs):
    return "\n\n".join([f"{i+1}. {d.metadata['title']} : {d.page_content}" for i,d in enumerate(docs)])

In [26]:
pineconeRetreiver = vectorstore.as_retriever(
        search_kwargs={'k' : 10}
)

candidate_books_qa = pineconeRetreiver | format_docs_title

candidate_books_qa.invoke(user_query)

'1. Adventures in Jamestown : In this revised edition, The Liberty Letters® series introduces fictional characters whose courage, ingenuity, and faith shaped events in U.S. history. Through the power of friendship, each story reveals how God works through ordinary teens in extraordinary times.<br />DARING YOUNG WOMEN—IN THE NEW WORLD AND THE OLD-- FIND MORE ADVENTURE THAN THEY BARGAINED FOR<br />After enduring a dangerous voyage to the New World, Abigail discovers that her fight for survival in Jamestown has only begun. When she must face her enemy, an Indian princess called Pocahontas, Abigail uncovers the enemy of her own heart—unforgiveness. In London, England, her friend Elizabeth yearns for adventure, but society’s conventions threaten to crush her dreams. As the girls face their deepest fears, they discover how their choices can change a nation’s---and a young woman’s—destiny.\n\n2. Masada: The Last Fortress : From the Back Cover:<br /><br />"In the year 72 C.E., after a four-yea

In [31]:
from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

In [40]:
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [42]:
chain = (
    {"candidate_books" : candidate_books_qa, 'user_information' : RunnablePassthrough()}
    | PromptTemplate.from_template(
        """
        You are a recommender. Based on the user's profile and behaviors,
        recommending a suitable book that she will like. Output the title of recommended book.
        
        User Information: 
        {user_information}

        Candidate Books:
        {candidate_books}

        Your response should be a list of comma separated values, eg: `foo, bar, baz`
        """
    )
    | llm
    | output_parser
)

In [43]:
chain.invoke(user_query)

['Masada: The Last Fortress']